Notes
    
        
- Heaviside step function
- def step_fun(x,a,b,c): return a * (np.sign(x-b) + c)
- p0=[np.max(y), np.median(x), np.min(y)]


In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit
from scipy.optimize import curve_fit
from mplcursors import cursor
print("done")

done


In [2]:
def step_sim(step_size = 8.0, SNR = 0.5, total_time_point=400):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(0, total_time_point), 
                            [
                                np.arange(0, total_time_point) < 99, 

                                np.logical_and(np.arange(0, total_time_point) >= 99, np.arange(0, total_time_point) < 199), 

                                np.logical_and(np.arange(0, total_time_point) >= 199, np.arange(0, total_time_point) < 249), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 249, np.arange(0, total_time_point) < 349), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 349, np.arange(0, total_time_point) < 499), 

                                np.arange(0, total_time_point) >= 499
                            ], 
                            [
                                lambda x: 300, 
                                lambda x: 100, 
                                lambda x: 50, 
                                lambda x: 350,
                                lambda x: 20,
                                lambda x: 350,
                            ])
    x = np.arange(0, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps
print("done")

done


In [3]:
from scipy.optimize import curve_fit
from dataclasses import dataclass

def moving_average(a, n=2, pad=False) :
    if pad:
        a = np.pad(a, (n-1,0))
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

@dataclass(repr=True)
class StepInfo:
    L:float
    x0:int
    k:float
    b:float
    location:float
    x:np.ndarray
    y:np.ndarray
    sqrt_error:np.ndarray
    

def _step_fun(x, L ,x0, k, b):
    # Logistic function
    # f(X, L, x0, k, b)
    # https://en.wikipedia.org/wiki/Logistic_function
    # use for curve fitting
    # https://stackoverflow.com/questions/55725139/fit-sigmoid-function-s-shape-curve-to-data-using-python
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return y

def get_step_location_size(x, y):
    p0=[np.max(y), np.median(x),1, np.min(y)]
    popt, pcov, infodict, mesg, ier = curve_fit(_step_fun, x, y,p0=p0, full_output=True, maxfev = 1000000)
    L , x0, k, b = popt
    sqrt_error = np.sqrt(infodict['fvec']**2)
    location = x[np.where(sqrt_error==np.min(sqrt_error))[0][0]]
    info = StepInfo(L , x0, k, b, location, x, y, sqrt_error)
    return info

def step_generator(x:np.ndarray, y:np.ndarray, steps:list, window_size_limit:int = 40):
    info = get_step_location_size(x, y)
    steps.append(info)
    location_index = np.where(x == info.location)[0][0]
    
    left_x = x[:location_index]
    left_y = y[:location_index]
    if len(left_y)>window_size_limit:
        step_generator(left_x, left_y, steps)
        
    right_x = x[location_index+1:]
    right_y = y[location_index+1:]
    if len(right_y)>window_size_limit:
        step_generator(right_x, right_y, steps)

# def step_generator(x:np.ndarray, y:np.ndarray, steps:list, counter_steps:list, is_counter=False, window_size_limit:int = 40):
#     if not is_counter:
#         info = get_step_location_size(x, y)
#         steps.append(info)
#     else:
#         info = get_step_location_size(x, y)
#         counter_steps.append(info)
    
#     if not is_counter:
#         location_index = np.where(x == info.location)[0][0]

#         left_x = x[:location_index]
#         left_y = y[:location_index]
#         if len(left_y)>window_size_limit:
#             step_generator(x=left_x, y=left_y, steps=steps, counter_steps=counter_steps, is_counter=False, window_size_limit=window_size_limit)

#         right_x = x[location_index+1:]
#         right_y = y[location_index+1:]
#         if len(right_y)>window_size_limit:
#             step_generator(x=right_x, y=right_y, steps=steps, counter_steps=counter_steps, is_counter=False, window_size_limit=window_size_limit )


#         counter_s =  np.where(x == left_x[len(left_x)//2])[0][0]
#         counter_e =  np.where(x == right_x[len(right_x)//2])[0][0]
#         counter_x = x[counter_s:counter_e]
#         counter_y = y[counter_s:counter_e]
#         if( len(counter_x) > window_size_limit):
#             step_generator(x=counter_x, y=counter_y, steps=steps, counter_steps=counter_steps, is_counter=True, window_size_limit=window_size_limit)
    
def get_y_from_step_locations(steps, y):
    # Generate the new steped data using the step locations and the orignal y data
    new_y = np.zeros(shape=y.shape)
    for index, s in enumerate(steps):
        if index == 0:
            new_y[0:s.location+1] = np.mean(y[0:s.location+1])
        else:
            pre_s = steps[index-1]
            cur_s = steps[index]
            new_y[pre_s.location +1:cur_s.location+1] = np.mean(y[pre_s.location +1:cur_s.location+1])
    
        if index == len(steps)-1:
            new_y[s.location +1:] = np.mean(y[s.location +1:])
    return new_y
print("done")

done


In [24]:
x_data, y_data , steps_org = step_sim(total_time_point=600)

step_info_list = []
step_generator(x_data, y_data, step_info_list)
step_info_list.sort(key=lambda x: x.location)


new_y = get_y_from_step_locations(step_info_list, y_data)

C:\Users\CellphyLab\AppData\Local\Temp\ipykernel_14404\3295757350.py:29: RuntimeWarning: overflow encountered in exp
  y = L / (1 + np.exp(-k*(x-x0))) + b


In [25]:
step_info_list_new = []

step_generator(x_data, new_y, step_info_list_new)
step_info_list_new.sort(key=lambda x: x.location)

new_y_2 = get_y_from_step_locations(step_info_list_new, new_y)

C:\Users\CellphyLab\AppData\Local\Temp\ipykernel_14404\3295757350.py:29: RuntimeWarning: overflow encountered in exp
  y = L / (1 + np.exp(-k*(x-x0))) + b


In [26]:
step_info_list_new = []

step_generator(x_data, new_y_2, step_info_list_new)
step_info_list_new.sort(key=lambda x: x.location)

new_y_3 = get_y_from_step_locations(step_info_list_new, new_y_2)

C:\Users\CellphyLab\AppData\Local\Temp\ipykernel_14404\3295757350.py:29: RuntimeWarning: overflow encountered in exp
  y = L / (1 + np.exp(-k*(x-x0))) + b


In [27]:
# w, k, y , m , c, r, g, b
plt.plot(x_data,  y_data,    label="noise")
plt.plot(x_data,  steps_org, label="raw")
plt.plot(x_data,  new_y,     label="new over fit")
plt.plot(x_data,  new_y_2,   label="new over fit_2")
plt.plot(x_data,  new_y_3,   label="new over fit_2")

plt.grid()
plt.legend()
cursor(hover=True)
plt.show()
print("done")

done


In [12]:
moving_avg_windows = 5
moving_avg_y = moving_average(y_data, n=moving_avg_windows, pad=True)

step_mv_info_list = []
moving_avg_x = np.arange(0, len(moving_avg_y))

plt.plot(moving_avg_x, moving_avg_y, '-g', label=f"mv {moving_avg_windows}")
step_generator(moving_avg_x, moving_avg_y, step_mv_info_list)
step_mv_info_list.sort(key=lambda x: x.location)

mv_new_y = get_y_from_step_locations(step_mv_info_list, moving_avg_y)

plt.plot(moving_avg_x, mv_new_y, '-r', label=f"mv {moving_avg_windows}")
plt.legend()
cursor(hover=True)
plt.show()

C:\Users\CellphyLab\AppData\Local\Temp\ipykernel_14404\3295757350.py:29: RuntimeWarning: overflow encountered in exp
  y = L / (1 + np.exp(-k*(x-x0))) + b


In [ ]:
x=np.array([ 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
       125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137,
       138, 139, 140, 141, 142, 143, 144, 145, 146])
y=np.array([127.9338085 , 108.19903961, 109.67804009, 105.45961098,
        93.02300008, 107.86276493, 120.86184236, 104.13556522,
       107.63626918, 106.50153184,  94.60625286, 104.14771736,
       109.16253162,  85.80643318,  81.9700607 , 100.39570474,
       103.84215196, 105.32569469, 113.34751294, 106.27081416,
        98.88421804, 108.48661795,  96.29020384,  85.4859683 ,
        95.12067085,  87.65852645,  90.55505851,  99.66192923,
        89.90461327,  88.18697181,  93.55911515, 102.30148861,
        98.90645805, 108.92554824, 119.2951933 , 105.35818738,
        87.9751338 ,  79.04569404,  81.73565712,  87.29630745,
        91.90870963,  92.31412478, 100.93253919,  99.96999548,
        92.68726807, 101.53842752, 102.84918664,  94.52567477])
plt.plot(x, y, '-r', label="test_data", linewidth=2)
# plt.plot(x_data, new_counter_y, '--r', label="new counter fit", linewidth=1)
plt.grid()
plt.legend()
cursor(hover=True)
plt.show()

In [ ]:
_stp=[]
# step_generator(x, y, _stp)
info = get_step_location_size(x, y)
print(info)
fit_y = _step_fun(x, info.step_size,info.location,info.stepness, info.b)
# plt.plot(x, fit_y, '-g', label="test_fit", linewidth=1)
sre = y - fit_y
print(sre)
print(np.min(sre))
print(np.where(sre == np.min(sre)))
print(x[14])

In [ ]:
print(y[14])

In [ ]:
sre_2 = np.sqrt((y - fit_y)**2)
print(sre_2)
print(np.min(sre_2))
print(np.where(sre_2 == np.min(sre_2)))
print(x[7])

In [ ]:
np.min(np.sqrt(np.array( [-11.34911102,5.98798899,2.38755878,4.72899922,15.50491867
,-0.80414862,-15.10318306,0.47297494,-4.04527297,-3.81078004
,7.28803714,-2.95806697,-8.59628154,14.20829206,17.5567301
,-1.30058842,-5.12893592,-6.95034304,-15.27106673,-8.45880623
,-1.30615525,-11.11552307,0.89778976,11.54003867,1.76202942
,9.09739306,6.08870052,-3.11739632,6.55213635,8.1921178
,2.75127034,-6.05188413,-2.71062516,-12.77728584,-23.18901542
,-9.28924068,8.06087536,16.96117606,14.24543435,8.6619783
,4.02940045,3.60613638,-5.02806853,-4.07949429,3.19087468
,-5.67121798,-6.99164943,1.32330557])**2))